### Reranking Hybrid Search Statergies

Re-ranking is a second-stage filtering process in retrieval systems, especially in RAG pipelines, where we:

1. First use a fast retriever (like BM25, FAISS, hybrid) to fetch top-k documents quickly.

2. Then use a more accurate but slower model (like a cross-encoder or LLM) to re-score and reorder those documents by relevance to the query.

👉 It ensures that the most relevant documents appear at the top, improving the final answer from the LLM.

In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser

d:\TrainingPython\Udemy\RAG_Pro\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## load text file
loader=TextLoader("langchain_sample.txt")
raw_docs=loader.load()

## Split text into document chunks
splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs=splitter.split_documents(raw_docs)
docs


[Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.\nBM25 is a traditional 

In [4]:
## user query
query="How can I use langchain to build an application with memory & tools"

In [5]:
# FAISS && HUggingFace model embeddings
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model=HuggingFaceEmbeddings(model_name="all-MinilM-L6-v2")
vector_store=FAISS.from_documents(docs,embedding_model)
retriever=vector_store.as_retriever(search_kwargs={"k":8})

In [6]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023AC7FB6900>, search_kwargs={'k': 8})

In [7]:
## Load LLM
import os 
from dotenv import load_dotenv
load_dotenv()



True

In [8]:
## prompt & use the llm
from langchain.chat_models import init_chat_model
opanAI_Groq=("openai/gpt-oss-120b")
llm = init_chat_model(
    model=opanAI_Groq,
    model_provider="OpenAI",
    api_key=os.getenv('GROQ_API_KEY'),
    base_url="https://api.groq.com/openai/v1",
    temperature=0.6
)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023ACB768830>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023ACB7692B0>, root_client=<openai.OpenAI object at 0x0000023ACB2B1E80>, root_async_client=<openai.AsyncOpenAI object at 0x0000023ACB769010>, model_name='openai/gpt-oss-120b', temperature=0.6, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')

In [10]:
## Prompt Template
prompt=PromptTemplate.from_template("""
You are helpful assistant. Your task is to rank the following documents from least relevant to the user's question.

User Question : "{question}"

Documents:
{documents}

Instructions:
- Think about relevance of each document to the user's question
- Return a list of the document indices in ranked order, starting from the most relevant

Output format: comma separated document indices (e.g: 2,1,3,0)
""")

In [11]:
retrieved_docs=retriever.invoke(query)
retrieved_docs

[Document(id='93b32325-2935-482f-9e7f-6010ebf3c8c7', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='97c0f0a1-d175-43aa-8e75-cf5d6a93b5e5', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='4819b62a-0853-4043-89cb-a7560872aa43', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging F

In [12]:
chain=prompt | llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\nYou are helpful assistant. Your task is to rank the following documents from least relevant to the user\'s question.\n\nUser Question : "{question}"\n\nDocuments:\n{documents}\n\nInstructions:\n- Think about relevance of each document to the user\'s question\n- Return a list of the document indices in ranked order, starting from the most relevant\n\nOutput format: comma separated document indices (e.g: 2,1,3,0)\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023ACB768830>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023ACB7692B0>, root_client=<openai.OpenAI object at 0x0000023ACB2B1E80>, root_async_client=<openai.AsyncOpenAI object at 0x0000023ACB769010>, model_name='openai/gpt-oss-120b', temperature=0.6, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='htt

In [13]:
doc_lines=[f"{i+1}. {doc.page_content}" for i,doc in enumerate(retrieved_docs)]
formatted_docs="\n".join(doc_lines)

In [14]:
doc_lines

['1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.',
 '2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.',
 '3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.',
 '4. Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground

In [15]:
formatted_docs

'1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.\n2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.\n3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.\n4. Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM respo

In [16]:
response=chain.invoke({"question": query, "documents": formatted_docs})
response

'1,5,2,3,4,6'

In [17]:
#step[5]: Parse & re-rank
indices=[int(x.strip())-1 for x in response.split(",") if x.strip().isdigit()]
indices

[0, 4, 1, 2, 3, 5]

In [18]:
retrieved_docs

[Document(id='93b32325-2935-482f-9e7f-6010ebf3c8c7', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='97c0f0a1-d175-43aa-8e75-cf5d6a93b5e5', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='4819b62a-0853-4043-89cb-a7560872aa43', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging F

In [20]:
reranked_docs=[retrieved_docs[i] for i in indices if 0<=i<len(retrieved_docs)]
reranked_docs

[Document(id='93b32325-2935-482f-9e7f-6010ebf3c8c7', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='3f843d6c-a5b4-4fd0-90b0-f6c271aa4d60', metadata={'source': 'langchain_sample.txt'}, page_content='FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed indexes, which makes it scalable for large document stores.\nAgents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering with search and code execution.'),
 Document(id='97c0f0a1-d175-43aa-8e75-cf5d6a93b5

In [21]:
#Step[6] : Show results
print("\n FInal Re-ranked Results: ")
for i,doc in enumerate(reranked_docs, 1):
    print(f"\nRank {i}: \n{doc.page_content}")



 FInal Re-ranked Results: 

Rank 1: 
LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.

Rank 2: 
FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed indexes, which makes it scalable for large document stores.
Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering with search and code execution.

Rank 3: 
LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt manag